In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import yfinance as yf

In [2]:
stock_tickers = set([
    # Large-cap stocks
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "BRK.B", "JPM", "JNJ", "V",
    "WMT", "PG", "MA", "UNH", "HD", "BAC", "XOM", "DIS", "CSCO", "VZ",
    
    # Growth stocks
    "TSLA", "SQ", "SHOP", "ROKU", "TTD", "DDOG", "CRWD", "ZS", "SNOW", "NET",
    "TWLO", "OKTA", "PLTR", "U", "RBLX", "ABNB", "DASH", "COIN", "AFRM", "RIVN",
    
    # Value stocks
    "CVX", "PFE", "MRK", "KO", "PEP", "T", "IBM", "GE", "F", "GM",
    "CAT", "MMM", "DOW", "CVS", "WBA", "MO", "PM", "BTI", "VLO", "PSX",
    
    # Low volatility stocks
    "PG", "JNJ", "KO", "PEP", "WMT", "MCD", "CL", "K", "GIS", "HSY",
    "CPB", "KMB", "CHD", "SJM", "HRL", "MKC", "CLX", "MDLZ", "KHC", "CAG",
    
    # Small-cap stocks
    "ATEN", "BCOV", "CAKE", "DENN", "EZPW", "FCEL", "GIII", "HLIT", "IRBT", "JBLU",
    "KMPR", "LGIH", "MGLN", "NAVI", "OFIX", "PLAB", "QADA", "RCII", "SANM", "TRUP",
    
    # Momentum stocks
    "AMD", "ENPH", "SEDG", "ETSY", "SNAP", "PINS", "ZM", "PTON", "BYND", "SPCE",
    "PLUG", "FSLR", "RUN", "BE", "NOVA", "CHPT", "BLNK", "EVGO", "LCID", "FSR",
    
    # Additional mixed stocks
    "AAL", "ABBV", "ABT", "ACN", "ADBE", "ADI", "ADM", "ADP", "AEP", "AIG",
    "ALK", "AMAT", "AMC", "AMD", "AME", "AMGN", "AMP", "AMT", "ANET", "ANSS",
    "AON", "AOS", "APA", "APD", "APH", "APTV", "ARE", "ATO", "ATVI", "AVB",
    "AVGO", "AVY", "AWK", "AXP", "AZO", "BA", "BAX", "BBY", "BDX", "BEN",
    "BF.B", "BG", "BIIB", "BIO", "BK", "BKNG", "BKR", "BLK", "BMY", "BR",
    "BRO", "BSX", "BWA", "BXP", "C", "CAH", "CB", "CBOE", "CCI", "CCL",
    "CDNS", "CDW", "CE", "CERN", "CF", "CFG", "CHD", "CHRW", "CHTR", "CI",
    "CINF", "CL", "CLX", "CMA", "CMCSA", "CME", "CMG", "CMI", "CMS", "CNC",
    "CNP", "COF", "COO", "COP", "COST", "CPB", "CPRT", "CRL", "CRM", "CSCO",
    "CSX", "CTAS", "CTLT", "CTSH", "CTVA", "CTXS", "CVS", "CVX", "CZR", "D",
    "DAL", "DD", "DE", "DFS", "DG", "DGX", "DHI", "DHR", "DIS", "DISH",
    "DLR", "DLTR", "DOV", "DOW", "DPZ", "DRE", "DRI", "DTE", "DUK", "DVA",
    "DVN", "DXC", "DXCM", "EA", "EBAY", "ECL", "ED", "EFX", "EIX", "EL",
    "EMN", "EMR", "ENPH", "EOG", "EQIX", "EQR", "ES", "ESS", "ETN", "ETR",
    "ETSY", "EVRG", "EW", "EXC", "EXPD", "EXR", "F", "FAST", "FB", "FBHS",
    "FCX", "FDS", "FDX", "FE", "FFIV", "FIS", "FISV", "FITB", "FLT", "FMC",
    "FOX", "FOXA", "FRC", "FRT", "FTNT", "FTV", "GD", "GE", "GILD", "GIS",
    "GL", "GLW", "GM", "GNRC", "GOOG", "GOOGL", "GPC", "GPN", "GRMN", "GS",
    "GWW", "HAL", "HAS", "HBAN", "HCA", "HD", "HES", "HIG", "HII", "HLT",
    "HOLX", "HON", "HPE", "HPQ", "HRL", "HSIC", "HST", "HSY", "HUM", "HWM",
    "IBM", "ICE", "IDXX", "IEX", "IFF", "ILMN", "INCY", "INTC", "INTU", "IP",
    "IPG", "IQV", "IR", "IRM", "ISRG", "IT", "ITW", "IVZ", "J", "JBHT",
    "JCI", "JKHY", "JNJ", "JNPR", "JPM", "K", "KEY", "KEYS", "KHC", "KIM",
    "KLAC", "KMB", "KMI", "KMX", "KO", "KR", "L", "LDOS", "LEG", "LEN",
    "LH", "LHX", "LIN", "LKQ", "LLY", "LMT", "LNC", "LNT", "LOW", "LRCX",
    "LUMN", "LUV", "LVS", "LW", "LYB", "LYV", "MA", "MAA", "MAR", "MAS",
    "MCD", "MCHP", "MCK", "MCO", "MDLZ", "MDT", "MET", "MGM", "MHK", "MKC",
    "MKTX", "MLM", "MMC", "MMM", "MNST", "MO", "MOH", "MOS", "MPC", "MPWR",
    "MRK", "MRNA", "MRO", "MS", "MSCI", "MSFT", "MSI", "MTB", "MTD", "MU",
    "NCLH", "NDAQ", "NEE", "NEM", "NFLX", "NI", "NKE", "NLOK", "NLSN", "NOC",
    "NOV", "NOW", "NRG", "NSC", "NTAP", "NTRS", "NUE", "NVDA", "NVR", "NWL",
    "NWS", "NWSA", "NXPI", "O", "ODFL", "OGN", "OKE", "OMC", "ORCL", "ORLY",
    "OTIS", "OXY", "PAYC", "PAYX", "PBCT", "PCAR", "PEAK", "PEG", "PENN", "PEP",
    "PFE", "PFG", "PG", "PGR", "PH", "PHM", "PKG", "PKI", "PLD", "PM",
    "PNC", "PNR", "PNW", "POOL", "PPG", "PPL", "PRGO", "PRU", "PSA", "PSX",
    "PTC", "PVH", "PWR", "PXD", "PYPL", "QCOM", "QRVO", "RCL", "RE", "REG",
    "REGN", "RF", "RHI", "RJF", "RL", "RMD", "ROK", "ROL", "ROP", "ROST",
    "RSG", "RTX", "SBAC", "SBUX", "SCHW", "SEE", "SHW", "SIVB", "SJM", "SLB",
    "SNA", "SNPS", "SO", "SPG", "SPGI", "SRE", "STE", "STT", "STX", "STZ",
    "SWK", "SWKS", "SYF", "SYK", "SYY", "T", "TAP", "TDG", "TDY", "TEL",
    "TER", "TFC", "TFX", "TGT", "TJX", "TMO", "TMUS", "TPR", "TRMB", "TROW",
    "TRV", "TSCO", "TSLA", "TSN", "TT", "TTWO", "TXN", "TXT", "TYL", "UA",
    "UAA", "UAL", "UDR", "UHS", "ULTA", "UNH", "UNM", "UNP", "UPS", "URI",
    "USB", "V", "VAR", "VFC", "VIAC", "VLO", "VMC", "VNO", "VRSK", "VRSN",
    "VRTX", "VTR", "VTRS", "VZ", "WAB", "WAT", "WBA", "WDC", "WEC", "WELL",
    "WFC", "WHR", "WLTW", "WM", "WMB", "WMT", "WRB", "WRK", "WST", "WU",
    "WY", "WYNN", "XEL", "XLNX", "XOM", "XRAY", "XYL", "YUM", "ZBH", "ZBRA",
    "ZION", "ZTS"
])

stocks_delisted = ['BF.B','NLOK', 'PEAK', 'XLNX', 'VAR', 'FB', 'PXD', 'RCII', 'CTXS', 'BRK.B', 'FSR', 'ATVI', 'WRK', 'FBHS', 'VIAC', 'PKI', 'DRE', 'QADA', 'CERN', 'WLTW', 'SIVB', 'PBCT', 'DISH', 'RE', 'MGLN', 'NLSN', 'FLT', 'FRC']

stock_tickers = [x for x in stock_tickers if x not in stocks_delisted]

In [3]:
value = yf.download(stock_tickers, period='1y')
value = value['Close'].reset_index()
value = value.rename_axis(None, axis=1)

[*********************100%***********************]  508 of 508 completed


In [4]:
## For S&P 500 index data.
ticker = "SPY"

# Download historical data
sp500_data = yf.download(ticker, period='1y')

sp500_data = sp500_data['Close'].reset_index()
sp500_data = sp500_data.rename_axis(None, axis=1)

[*********************100%***********************]  1 of 1 completed


In [5]:
sp500_data.to_csv('SPY_1yr.csv',index=False)
value.to_csv('500_stocks_1yr.csv',index=False)